1. unsupervised pretraining
2. SFT
3. DPO
4. Inferencing

# Install Libraries

In [ ]:
!pip install transformers trl peft accelerate datasets bitsandbytes adapters--quiet

  Using cached trl-0.16.0-py3-none-any.whl.metadata (12 kB)
  Using cached datasets-3.4.1-py3-none-any.whl.metadata (19 kB)
  Using cached bitsandbytes-0.45.3-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
ERROR: Could not find a version that satisfies the requirement adapters--quiet (from versions: none)
ERROR: No matching distribution found for adapters--quiet


In [ ]:
pip install adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 117.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0


# logging to huggnigface

In [ ]:
##from huggingface_hub import notebook_login
#notebook_login()

# Import Required Libraries

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments

In [ ]:
from datasets import load_dataset, Dataset

In [ ]:
from trl import SFTTrainer, DPOConfig, DPOTrainer

In [ ]:
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model

In [ ]:
import torch

In [ ]:
from accelerate import Accelerator

In [ ]:
pip install trl --upgrade --quiet

## Load Pre-trained Model

In [ ]:
# Constructor: Initializes the dataset and configuration parameters.

# Prepares the LoRA model configuration for fine-tuning.
# Ensures 'self.model' is defined before calling 'get_peft_model' to avoid errors.

# Loads the pre-trained model with 4-bit precision for memory efficiency.
# Maps the model to the correct device (CPU/GPU).
# Disables cache to allow gradient updates during training.
# Prepares the model for low-bit fine-tuning using LoRA if enabled in the configuration.

# Loads the tokenizer and sets the pad token.
# Setting pad_token as eos_token is a common practice, but check if it aligns with your dataset needs.

# Defines training arguments such as batch size, optimizer, learning rate, and number of training epochs.
# Uses 'fp16' for mixed precision training to optimize performance.
# Disables logging to external platforms using 'report_to="none"'.

# Creates the trainer after loading the model and tokenizer.
# Prints trainable parameters when using LoRA for debugging.
# Initializes the trainer with model, dataset, and training arguments.

# Starts the training process.
# Saves the fine-tuned model and tokenizer in the specified output directory.
# Ensures the tokenizer is saved properly for later inference.

In [ ]:
## parameter for the SFT
sft_config={
    "model_ckpt": "TinyLlama/TinyLlama-1.1B-Chat-v1.0", #This is my Pretrain model
            "load_in_4bit": True,
            "device_map": {"": Accelerator().local_process_index},
            "torch_dtype": torch.float16,
            "torch_dtype": torch.float16,
            "trust_remote_code": True,

            "use_lora": True,
            "r": 16,
            "lora_alpha": 16,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj", "v_proj"],

            "output_dir": "sft-tiny-llama-chatbot",
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit",
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 100,
            "num_train_epochs": 1,
            "max_steps": 250,
            "fp16": True,
            "push_to_hub": True,
            "packing": False,
            "max_seq_length": 512,
            "neftune_noise_alpha": 5
}


In [ ]:
class TrainSFT:
  def __init__(self,data,config) -> None:
    self.data=data
    self.config=config

  def prepare_lora_model(self):
    self.lora_config=LoraConfig(r=self.config["r"],
                                lora_alpha=self.config["lora_alpha"],
                                lora_dropout=self.config["lora_dropout"],
                                bias=self.config["bias"],
                                task_type=self.config["task_type"],
                                target_modules=self.config["target_modules"])

    self.model=get_peft_model(self.model,self.lora_config)

  def load_model_tokenizer(self):
    self.model=AutoModelForCausalLM.from_pretrained(
                              self.config["model_ckpt"],
                              load_in_4bit=self.config["load_in_4bit"],
                              device_map=self.config["device_map"],
                              torch_dtype=self.config["torch_dtype"]
      )
    self.model.config.use_cache=False
    self.model.config.pretraining_tp=1
    self.model = prepare_model_for_kbit_training(self.model)

    if self.config["use_lora"]:
      self.prepare_lora_model()

    self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
    self.tokenizer.pad_token = self.tokenizer.eos_token
    return self.tokenizer

  def set_training_args(self):
      return TrainingArguments(
                              output_dir=self.config["output_dir"],
                              per_device_train_batch_size=self.config["per_device_train_batch_size"],
                              gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
                              optim=self.config["optim"],
                              learning_rate=self.config["learning_rate"],
                              lr_scheduler_type=self.config["lr_scheduler_type"],
                              save_strategy=self.config["save_strategy"],
                              logging_steps=self.config["logging_steps"],
                              num_train_epochs=self.config["num_train_epochs"],
                              max_steps=self.config["max_steps"],
                              fp16=self.config["fp16"],
                              push_to_hub=self.config["push_to_hub"],
                              neftune_noise_alpha=self.config["neftune_noise_alpha"],
                              report_to="none",
                              #dataset_text_field="text",  ##
                                )

  def create_trainer(self):
    self.tokenizer = self.load_model_tokenizer()
    if self.config["use_lora"]:
      print(self.model.print_trainable_parameters())
      self.trainer = SFTTrainer(
                        model=self.model,
                        train_dataset=self.data,
                        peft_config=self.lora_config,
                        args=self.set_training_args(),
                        processing_class=self.tokenizer,
                        #dataset_text_field="text"
                        )


  def train_and_save_model(self):
    self.create_trainer()
    self.trainer.train()
    self.trainer.save_model(self.config["output_dir"])
    self.tokenizer.save_pretrained(self.config["output_dir"])


In [ ]:
def create_data():
  data=load_dataset("tatsu-lab/alpaca", split="train")
  data_df = data.to_pandas()
  data_df = data_df[:700]
  data_df["text"] = data_df[["input", "instruction", "output"]].apply(lambda x: "Human: " + x["instruction"] + " " + x["input"] + " Assistant: "+ x["output"], axis=1)
  data = Dataset.from_pandas(data_df)
  return data

In [ ]:
data=create_data()

In [ ]:
data

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 700
})

In [ ]:
data[0]

{'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
 'text': 'Human: Give three tips for staying healthy.  Assistant: 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}

In [ ]:
train_sft=TrainSFT(data,sft_config)

In [ ]:
train_sft.train_and_save_model()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044
None


Converting train dataset to ChatML:   0%|          | 0/700 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/700 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/700 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/700 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/pyt

Step,Training Loss
100,2.111500
200,1.712800


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Prefrence Alignment- DPO

Will train or finetune our model using PEFT(SFT) then will retrain for prefrence alignment for controlled response using DPO

In [ ]:
dpo_config = {
            "model_ckpt": "Abhimanyu9539/sft-tiny-llama-chatbot",
            "load_in_4bit": True,
            "device_map": {"": Accelerator().local_process_index},
            "torch_dtype": torch.float16,
            "trust_remote_code": True,
            "use_lora": True,
            "r": 8,
            "lora_alpha": 8,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj", "v_proj"],
            "output_dir": "tiny-llama-dpo",
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit",
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 100,
            "num_train_epochs": 1,
            "max_steps": 250,
            "fp16": True,
            "push_to_hub": True,
            "train_cln_name": "text",
            "packing": False,
            "neftune_noise_alpha": 5,
            "beta": 0.1,
            "loss_type": "kto_pair",
            "max_length": 768,
            "max_prompt_length": 512,
            "max_target_length": 256,
            "is_encoder_decoder": False
        }

### DPO optimizes a policy (trainable model) by comparing its outputs against a reference model:

In [ ]:
class TrainDPO:
  def __init__(self,data,config) -> None:
    self.data=data
    self.config=config

  def prepare_lora_model(self):
    self.lora_config = LoraConfig(
                                r=self.config["r"],
                                lora_alpha=self.config["lora_alpha"],
                                lora_dropout=self.config["lora_dropout"],
                                bias=self.config["bias"],
                                task_type=self.config["task_type"],
                                target_modules=self.config["target_modules"]
                                )
    self.model = get_peft_model(self.model, self.lora_config) #THIS MODEL WHICH I AM GOING TO TRAIN FURTHER ON MY HUMAN PREFRENCE DATA
    self.model_ref = get_peft_model(self.model_ref, self.lora_config) #SAME MODEL IN THIS VARIABLE FOR THE REFRENCE #FROZAN WEIGHTS



  def load_model_tokenizer(self):
        self.model = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"]
                        )

        self.model_ref = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"]
                        )
        self.model.config.use_cache=False
        self.model.config.pretraining_tp=1
        self.model = prepare_model_for_kbit_training(self.model)
        if self.config["use_lora"]:
            self.prepare_lora_model()

        self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
        self.tokenizer.pad_token = self.tokenizer.eos_token

  def set_training_args(self):
    return DPOConfig(
                        output_dir=self.config["output_dir"],
                        per_device_train_batch_size=self.config["per_device_train_batch_size"],
                        gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
                        optim=self.config["optim"],
                        learning_rate=self.config["learning_rate"],
                        lr_scheduler_type=self.config["lr_scheduler_type"],
                        save_strategy=self.config["save_strategy"],
                        logging_steps=self.config["logging_steps"],
                        num_train_epochs=self.config["num_train_epochs"],
                        max_steps=self.config["max_steps"],
                        fp16=self.config["fp16"],
                        push_to_hub=self.config["push_to_hub"],
                        neftune_noise_alpha=self.config["neftune_noise_alpha"],
                        report_to="none",
                        remove_unused_columns=False,
                        gradient_checkpointing=True,  # Enable gradient checkpointing
                        gradient_checkpointing_kwargs={'use_reentrant': True}

                            )

  def create_trainer(self):
    self.load_model_tokenizer()

    if self.config["use_lora"]:
        print(self.model.print_trainable_parameters())
        self.trainer = DPOTrainer(
                                  self.model,
                                  self.model_ref,
                                  args=self.set_training_args(),
                                  train_dataset=self.data,
                                  processing_class=self.tokenizer,

                                )

  def train_and_save_model(self):
    self.create_trainer()
    self.trainer.train()
    self.trainer.save_model(self.config["output_dir"])
    self.tokenizer.save_pretrained(self.config["output_dir"])




In [ ]:
def create_data():
    df = load_dataset("Anthropic/hh-rlhf", split="train").to_pandas()
    df["prompt"] = df["chosen"].apply(lambda x: x.split("Assistant: ")[0])
    df["chosen"] = df["chosen"].apply(lambda x: "Assistant: "+ x.split("Assistant: ")[-1])
    df["rejected"] = df["rejected"].apply(lambda x: "Assistant: " + x.split("Assistant: ")[-1])
    df = df.sample(1000)
    data = Dataset.from_pandas(df)
    return data

In [ ]:
data=create_data()

README.md:   0%|          | 0.00/5.77k [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

...

test.jsonl.gz:   0%|          | 0.00/875k [00:00<?, ?B/s]

test.jsonl.gz:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

test.jsonl.gz:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/160800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8552 [00:00<?, ? examples/s]

In [ ]:
data

Dataset({
    features: ['chosen', 'rejected', 'prompt', '__index_level_0__'],
    num_rows: 1000
})

In [ ]:
data[0]

{'chosen': 'Assistant: I’m sorry, I’m not sure what I could clarify here.  The Normandy Empire eventually became the Kingdom of England, which is what you see as England today.  That was in 1066.  Is there anything else I could clarify?',
 'rejected': 'Assistant: Which question was that?  I’m not sure what else you might be asking, so I’m hoping you’ll let me know.',
 'prompt': '\n\nHuman: At what point in history did Christianity replace all other religions in Western Europe?\n\n',
 '__index_level_0__': 59219}

In [ ]:
data[0]["chosen"]

'Assistant: I’m sorry, I’m not sure what I could clarify here.  The Normandy Empire eventually became the Kingdom of England, which is what you see as England today.  That was in 1066.  Is there anything else I could clarify?'

In [ ]:
data[0]["rejected"]

'Assistant: Which question was that?  I’m not sure what else you might be asking, so I’m hoping you’ll let me know.'

In [ ]:
data[0]["prompt"]

'\n\nHuman: At what point in history did Christianity replace all other religions in Western Europe?\n\n'

In [ ]:
train_dpo=TrainDPO(data,dpo_config)

In [ ]:
train_dpo.train_and_save_model()

adapter_config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


adapter_model.safetensors:   0%|          | 0.00/9.02M [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

...

Applying chat template to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recom

Step,Training Loss
100,0.693700
200,0.710400


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


### This Inferencing part is homework(assisgnment)

#### load the model from huggingface

#### fit into the transformer pipeline

#### then make a prediction out of it

In [ ]:
from transformers import pipeline

question = "How do you embezzle money?"
generator = pipeline("text-generation", model="Abhimanyu9539/tiny-llama-dpo", device="cuda")
output = generator([{"role": "user", "content": question}], max_new_tokens=128, return_full_text=False)[0]
print(output["generated_text"])


Device set to use cuda


Embezzlement is the theft or misappropriation of money or property by an individual or an organization. Here are some ways in which someone can embezzle money:

1. Forgery: A person can forge a check or a bank statement to make it appear that they have received money that they did not actually receive.

2. Fraud: A person can create a false account or a fake bank account to steal money from the organization or individual.

3. Misappropriation: A person can misuse the money or property for their own benefit, such as using


In [ ]:
question = "How do you embezzle money?"
generator = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", device="cuda")
output = generator([{"role": "user", "content": question}], max_new_tokens=128, return_full_text=False)[0]
print(output["generated_text"])


Device set to use cuda


Embezzlement is the theft or misappropriation of money or property by an individual or an organization. Here are some ways in which someone can embezzle money:

1. Forgery: A person can forge a check or a bank statement to make it appear that they have received money that they did not actually receive.

2. Fraud: A person can create a false account or a fake bank account to steal money from the organization or individual.

3. Misappropriation: A person can misuse the money or property for their own benefit, such as using


In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model_name = "Abhimanyu9539/sft-tiny-llama-chatbot"
adapter_model_name = "Abhimanyu9539/tiny-llama-dpo"

base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
model = PeftModel.from_pretrained(base_model, adapter_model_name)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

adapter_config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/9.02M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline


In [ ]:
# Prepare input
prompt = "How to rob an atm?"
inputs = tokenizer(prompt, return_tensors="pt")

# Move both input and the model to the same device
device = "cuda:0"  # Or 'cpu' if you don't have a GPU
inputs = inputs.to(device)
model = model.to(device) # Move the PEFT model to the GPU

# Generate text
output = model.generate(**inputs, max_length=100)

# Decode the output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

How to rob an atm?
How to rob an ATM?
How to rob an ATM?
How to rob an ATM?
How to rob an ATM?
How to rob an ATM?
How to rob an ATM?
How to rob an ATM?
How to rob an ATM?
How to rob an ATM?
How to rob an ATM?
How to rob an ATM?
How to rob
